In [2]:
# Translated to .py by Meritxell Pacheco
# 2017
# Adapted to PandasBiogeme by Michel Bierlaire
# Sun Oct 21 23:15:31 2018

import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.expressions import Beta, DefineVariable
from biogeme.models import loglogit

pandas = pd.read_table("lpmc02.dat")
database = db.Database("LPMC",pandas)
pd.options.display.float_format = '{:.3g}'.format

globals().update(database.variables)

# Exclude
# exclude = (  ArrivalTimeHours_1   ==  -1  )
# database.remove(exclude)
  
# Choice
chosenAlternative = travel_mode

#Parameters to be estimated+ (  BestAlternative_4   *  4  )
# Arguments:
#   1  Name for report. Typically, the same as the variable
#   2  Starting value
#   3  Lower bound
#   4  Upper bound
#   5  0: estimate the parameter, 1: keep it fixed
Constant1	 = Beta('Constant1',0,None,None,1)
Constant2	 = Beta('Constant2',0,None,None,0)
Constant3	 = Beta('Constant3',0,None,None,0)
Constant4	 = Beta('Constant4',0,None,None,0)
Cost	 = Beta('Cost',0,None,None,0)
Total_TT1	 = Beta('Total_TT1',0,None,None,0)
Total_TT2	 = Beta('Total_TT2',0,None,None,0)
Total_TT3	 = Beta('Total_TT3',0,None,None,0)
Total_TT4	 = Beta('Total_TT4',0,None,None,0)
DrivLicense	 = Beta('Drivlicense',0,None,None,0)
TrainCard	 = Beta('TrainCard',0,None,None,0)
BusCard	 = Beta('BusCard',0,None,None,0)

# Define here arithmetic expressions for name that are not directly
# available from the data

dur_public = DefineVariable('dur_public', dur_pt_access + dur_pt_rail + dur_pt_bus + dur_pt_int ,database)
cost_public = DefineVariable('cost_public', cost_transit ,database)
cost_driving = DefineVariable('cost_driving', cost_driving_fuel + cost_driving_ccharge ,database)


# Utilities

#Opt1 = walking
#Opt2 = cycling
#Opt3 = public transport
#Opt4 = driving

Opt1 = Constant1 + Total_TT1 * dur_walking
Opt2 = Constant2 + Total_TT2 * dur_cycling
Opt3 = Constant3 + Cost * cost_public + Total_TT3 * dur_public + TrainCard * faretype + BusCard * bus_scale
Opt4 = Constant4 + Cost * cost_driving + Total_TT4 * dur_driving + DrivLicense * driving_license
V = {1: Opt1,2: Opt2,3: Opt3,4: Opt4}
av = {1: 1,2: 1,3: 1,4: 1}

# The choice model is a logit, with availability conditions
logprob = loglogit(V,av,chosenAlternative)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_lpmc_model2_licenses"
results = biogeme.estimate()
# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)
print(f"Nbr of observations: {database.getNumberOfObservations()}")
print(f"LL(0) =    {results.data.initLogLike:.3f}")
print(f"LL(beta) = {results.data.logLike:.3f}")
print(f"rho bar square = {results.data.rhoBarSquare:.3g}")
print(f"Output file: {results.data.htmlFileName}")

              Value  Std err  t-test  p-value  Rob. Std err  Rob. t-test  \
BusCard      0.0108    0.142  0.0762    0.939         0.141       0.0769   
Constant2     -4.71    0.175   -26.9        0         0.223        -21.1   
Constant3     -2.27    0.205   -11.1        0         0.227          -10   
Constant4      -2.8    0.121   -23.2        0         0.166        -16.8   
Cost         -0.148   0.0157   -9.38        0        0.0149        -9.91   
Drivlicense    1.43   0.0723    19.8        0        0.0719         19.9   
Total_TT1     -8.44    0.299   -28.2        0         0.471        -17.9   
Total_TT2     -6.05    0.455   -13.3        0         0.531        -11.4   
Total_TT3     -3.87    0.247   -15.7        0         0.254        -15.2   
Total_TT4     -7.08    0.351   -20.2        0         0.396        -17.9   
TrainCard   -0.0244   0.0376  -0.648    0.517        0.0377       -0.647   

             Rob. p-value  
BusCard             0.939  
Constant2               0  
Con